In [1]:
## model modules for total detection model

"""
*module 1: part_symptom_classifying.h5

    * x_data: part image features: bboxes for each plant image -> extract features
    * y_data: part_status(Y: yes symptom, N: no symptom)
    * model pulpose: regardless which is in a certain part of plant, distinguish the status of input par

*module 2: part_disease_class_classifying.h5

    * x_data: same as module 1 but only use 'Y' labeled parts
    * y_data: disease_class data
    * module purpose: regardless which is in a certain part, distinguish the disease class of 'Y' labeled parts

* module 1, 2 integrated structure(after modules trained)(ex. blight)
    *input data: image[bbox1, bbox2,...] -> 이 리스트를 순회하면서 개별 바운딩 박스에 대해 모듈 추론 적용후 결과 병합)
        *module 1 prediction(input: image[])
            *prediction: image[N, Y, N, Y, ...]
            *classifying: 
                if all image[i] == 'N' --> {normal(output directly and start next loop)}
                else                   --> abnormal(go to module 2)

                *module 2 prediction(input: abnormal([N, Y, N, Y])
                    output_list = []
                    *classifying:
                        if abnormal[] == 'N' --> output list.append() 
                        else                 --> output_list.append(predicted_class)

    final_output:
        *(normal): imagename = {bbox: [[bbox1], [bbox2],...], status: [N, N, ...], class: normal}
        *(abnormal): imagename = {bbox: [[bbox1], [bbox2], ...], status: [N, Y, N, Y], class: blight}

"""

"\n*module 1: part_symptom_classifying.h5\n\n    * x_data: part image features: bboxes for each plant image -> extract features\n    * y_data: part_status(Y: yes symptom, N: no symptom)\n    * model pulpose: regardless which is in a certain part of plant, distinguish the status of input par\n\n*module 2: part_disease_class_classifying.h5\n\n    * x_data: same as module 1 but only use 'Y' labeled parts\n    * y_data: disease_class data\n    * module purpose: regardless which is in a certain part, distinguish the disease class of 'Y' labeled parts\n\n* module 1, 2 integrated structure(after modules trained)\n    *input data: image[bbox1, bbox2,...]\n    *module 1 prediction(input: image[])\n        *prediction: image[N, Y, N, Y, ...]\n        *classifying: \n            if all image[i] == 'N' --> {normal(prediction finished)}\n            else                   --> abnormal(go to module 2)\n\n            *module 2 prediction(input: abnormal([N, Y, N, Y])\n                output_list = []

In [1]:
# import lists

import pandas as pd
import numpy as np
import os
import glob
import torch
from torchvision import models, transforms
from torchvision.models import Inception_V3_Weights
from PIL import Image
import cv2
from tqdm import tqdm

In [32]:
test_data = pd.read_csv('./data_preprocessed/label/test_label_10.csv', encoding='utf-8')
test_data_2 = pd.read_csv('./data_preprocessed/label/test_label_second.csv', encoding='utf-8')
train_10 = pd.read_csv('./data_preprocessed/label/train_label_10.csv', encoding='utf-8')
train_50 = pd.read_csv('./data_preprocessed/label/train_label_50.csv', encoding='utf-8')
train_70 = pd.read_csv('./data_preprocessed/label/train_label_70.csv', encoding='utf-8')

In [44]:
# image_width = train_20['image_width']
# image_height = train_20['image_height']
# x = train_20['bbox_x']
# y = train_20['bbox_y']
# width = train_20['bbox_width']
# height = train_20['bbox_height']

# width_same = train_20[image_width == (x+width)]
# height_same = train_20[image_height == (y+height)]
# width_height_same = train_20[(image_width == (x+width)) & (image_height == (y+height))]

# print(len(width_same))
# print(len(height_same))
# print(len(width_height_same))
# print(width_same.head(1))

0
0
0
Empty DataFrame
Columns: [file_id, folder_path, file_name, image_width, image_height, class, cause_method, ei_value, pl_value, el_value, pi_value, bbox, bbox_x, bbox_y, bbox_width, bbox_height, category_id, disease_status, name, object_status]
Index: []


In [12]:
# part image extraction and image preprocess for CNN model

class PartImageExtractor:
    def __init__(self):
        self.transform = transforms.Compose([
            transforms.Resize(299),
            transforms.CenterCrop(299),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        self.model = models.inception_v3(weights=Inception_V3_Weights.DEFAULT)
        self.model.eval()

        if torch.cuda.is_available():
            self.model = self.model.cuda()

    def extract_features(self, image, bbox):
        x, y, w, h = bbox
        image = Image.open(image)
        image = image.crop((x, y, x+w, y+h))
        image = self.transform(image)
        image = image.unsqueeze(0)

        if torch.cuda.is_available():
            image = image.cuda()

        with torch.no_grad():
            features = self.model(image)

        return features.cpu().numpy()
    
    def extract_features_from_array(self, cropped_img_array):
        image = Image.fromarray(np.uint8(cropped_img_array))
        image = self.transform(image)
        image = image.unsqueeze(0)

        if torch.cuda.is_available():
            image = image.cuda()

        with torch.no_grad():
            features = self.model(image)
        
        return features.cpu().numpy()


In [ ]:
extractor = PartImageExtractor()
features_list = []

for index, row in train_10.iterrows():
    folder_path = row['folder_path']
    file_name = row['file_name']
    bbox = (row['bbox_x'], row['bbox_y'], row['bbox_width'], row['bbox_height'])

    image_path = os.path.join(folder_path, file_name)
    features = extractor.extract_features(image_path, bbox)
    features_list.append(features)

features_array = np.array(features_list)
print(features_array.shape)

In [ ]:
np.savez("./data_preprocessed/part_image_features/cropped_features_10.npz", *features_array)

In [71]:
extractor = PartImageExtractor()
features_dict = {}

for index, row in tqdm(train_10.iterrows(), total=len(train_10), desc="Processing images"):
    folder_path = row['folder_path']
    file_name = row['file_name']
    bbox = (row['bbox_x'], row['bbox_y'], row['bbox_width'], row['bbox_height'])

    image_path = os.path.join(folder_path, file_name)
    features = extractor.extract_features(image_path, bbox)
    
    features_id = row['bbox_id']
    features_dict[features_id] = features

print(features_dict[train_10['bbox_id'][0]].shape)
print(len(features_dict))


Processing images: 100%|██████████| 28382/28382 [20:16<00:00, 23.33it/s]

(1, 1000)
28382


In [72]:
np.savez("./data_preprocessed/part_image_features/cropped_features_10_dict.npz", **features_dict)

In [73]:
train_10_y = train_10[train_10['disease_status'] == 'Y'].reset_index(drop=True).copy()
print(len(train_10), len(train_10_y))

28382 4510


In [75]:
extractor = PartImageExtractor()
features_y_dict = {}

for index, row in tqdm(train_10_y.iterrows(), total=len(train_10_y), desc="Processing images"):
    folder_path = row['folder_path']
    file_name = row['file_name']
    bbox = (row['bbox_x'], row['bbox_y'], row['bbox_width'], row['bbox_height'])

    image_path = os.path.join(folder_path, file_name)
    features = extractor.extract_features(image_path, bbox)
    features_y_id = row['bbox_id']
    features_y_dict[features_y_id] = features

print(features_y_dict[train_10_y['bbox_id'][0]].shape)
print(len(features_y_dict))

Processing images: 100%|██████████| 4510/4510 [03:11<00:00, 23.54it/s]

(1, 1000)
4510


In [76]:
np.savez("./data_preprocessed/part_image_features/cropped_features_10_y_dict.npz", **features_y_dict)

In [77]:
test_copy = test_data[(test_data['bbox_width'] > 1) & (test_data['bbox_height'] > 1)].copy()

print(len(test_copy))
print(len(test_data))

17864
17864


In [37]:
test_copy = test_data_2.copy().reset_index(drop=True)
test_copy['vl_id'] = None
for idx, _ in test_copy.iterrows():
    test_copy.at[idx, 'vl_id'] = f"val_{idx}"

print(len(test_copy))
print(test_copy['vl_id'][0])

10106
val_0


In [64]:
# test_copy_2 = test_data_2.copy().reset_index(drop=True)
original_test_data = pd.read_csv('./data_preprocessed/label/test_label_third.csv', encoding='utf-8')
test_copy = original_test_data.copy()

test_normal = test_copy[test_copy['file_id'].str.contains('normal')].reset_index(drop=True)
test_blight = test_copy[test_copy['file_id'].str.contains('blight')].reset_index(drop=True)
test_wilt = test_copy[test_copy['file_id'].str.contains('wilt')].reset_index(drop=True)
test_scorch = test_copy[test_copy['file_id'].str.contains('scorch')].reset_index(drop=True)
test_chlorosis = test_copy[test_copy['file_id'].str.contains('chlorosis')].reset_index(drop=True)

test_list = [test_normal, test_blight, test_wilt, test_scorch, test_chlorosis]
id_list = ['normal', 'blight', 'wilt', 'scorch', 'chlorosis']

results = []
for df, id in zip(test_list, id_list):
    df['val_id'] = None
    data_id = id
    for idx, _ in df.iterrows():
        df.at[idx, 'val_id'] = f"val_{id}_{idx}"

result_df = pd.concat(test_list, axis=0).reset_index(drop=True)


In [67]:
# result_df = result_df.drop(['vl_id'], axis=1)
blight_check = result_df[result_df['val_id'].str.contains('blight')].reset_index(drop=True)

print(blight_check['val_id'][0])

val_blight_0


In [55]:
print(len(result_df))
result_sample = result_df.iloc[:][:100].copy()
print(len(result_sample))
print(result_sample.columns)

10106
100
Index(['file_id', 'folder_path', 'file_name', 'image_width', 'image_height',
       'class', 'cause_method', 'ei_value', 'pl_value', 'el_value', 'pi_value',
       'bbox', 'bbox_x', 'bbox_y', 'bbox_width', 'bbox_height', 'category_id',
       'disease_status', 'name', 'object_status', 'bbox_id', 'sampling_id',
       'val_id'],
      dtype='object')


In [59]:
original_test_data = pd.read_csv('./data_preprocessed/label/test_label_third.csv', encoding='utf-8')
print(len(original_test_data))

11082


In [68]:
extractor = PartImageExtractor()
vl_features_dict = {}

for index, row in tqdm(result_df.iterrows(), total=len(result_df), desc="Processing images"):
    folder_path = row['folder_path']
    file_name = row['file_name']
    bbox = (row['bbox_x'], row['bbox_y'], row['bbox_width'], row['bbox_height'])

    image_path = os.path.join(folder_path, file_name)
    features = extractor.extract_features(image_path, bbox)
    features_id = row['val_id']
    vl_features_dict[features_id] = features

print(vl_features_dict[result_df['val_id'][0]].shape)
print(len(vl_features_dict))

Processing images: 100%|██████████| 11082/11082 [06:54<00:00, 26.76it/s]


(1, 1000)
11082


In [ ]:
## 이미지 추출 및 딕셔너리화 다시
    ## 기존 문제: bbox_id를 딕셔너리 키로 할 경우 라벨df와 특징 딕셔너리의 길이 불일치 문제 발생
    ## 대응책: bbox_id컬럼 삭제, tr_id, val_id로 대체(인덱싱 규칙은 위에서 작업한것 참고)
## 특징 추출 최종 데이터셋
    ## 훈련데이터: 10, 50, 100
    ## 검증데이터: 그대로

In [39]:
np.savez("./data_preprocessed/part_image_features/vl_cropped_features_10_dict.npz", **vl_features_dict)
# test_copy.to_csv('./data_preprocessed/label/test_label_second.csv', encoding='utf-8', index=False)

In [53]:
extractor = PartImageExtractor()
vl_features_list = []

for index, row in tqdm(test_copy.iterrows(), total=len(test_copy), desc="Processing images"):
    folder_path = row['folder_path']
    file_name = row['file_name']
    bbox = (row['bbox_x'], row['bbox_y'], row['bbox_width'], row['bbox_height'])

    image_path = os.path.join(folder_path, file_name)
    features = extractor.extract_features(image_path, bbox)
    vl_features_list.append(features)

vl_features_array = np.array(vl_features_list)
print(vl_features_array.shape)

Processing images: 100%|██████████| 18045/18045 [11:46<00:00, 25.56it/s]

(18045, 1, 1000)


In [54]:
np.savez("./data_preprocessed/part_image_features/vl_cropped_features_10.npz", *vl_features_array)

In [12]:
test_npz_files = np.load('./data_preprocessed/part_image_arrays/vl_cropped.npz')

cropped_image_arrays_list = []
test_cropped_features = []

for key in test_npz_files.files:
    cropped_img_array = test_npz_files[key]
    cropped_image_arrays_list.append(cropped_img_array)



In [25]:
extractor = PartImageExtractor()
error_indices = []

for index, test_cropped in enumerate(tqdm(cropped_image_arrays_list, total=len(cropped_image_arrays_list), desc="Extracting features")):
    try:

        features = extractor.extract_features_from_array(test_cropped)
        test_cropped_features.append(features)
    except Exception as e:
        print(f"Error processing image at index {index}: {e}")
        error_indices.append(index)
        continue

test_cropped_features_arrays = np.array(test_cropped_features)
print(test_cropped_features_arrays.shape)
print("Error indices:", error_indices)

Extracting features:  17%|█▋        | 30358/181343 [06:09<29:30, 85.29it/s]

Error processing image at index 30347: division by zero


Extracting features:  36%|███▌      | 64889/181343 [13:05<23:34, 82.32it/s] 

Error processing image at index 64872: division by zero


Extracting features: 100%|██████████| 181343/181343 [36:20<00:00, 83.15it/s]


(211688, 1, 1000)
Error indices: [30347, 64872]


In [29]:
print(len(cropped_image_arrays_list))
print(len(test_cropped_features_arrays))

181343
211688


In [38]:
print(test_data.iloc[30347])
print(test_data.iloc[64872])

file_id                               vl_normal_6622
folder_path       ./data_raw/test_image/test_normal/
file_name             normal_58_003_221127161748.jpg
image_width                                     1552
image_height                                    2106
class                                         normal
cause_method                                  normal
folder_path.1     ./data_raw/test_image/test_normal/
ei_value                                        0.03
pl_value                                         6.0
el_value                                         1.2
pi_value                                         5.7
bbox                  [1127.05, 1316.61, 0.47, 0.47]
category_id                                        4
disease_status                                     N
name                                            stem
bbox_x                                       1127.05
bbox_y                                       1316.61
bbox_width                                    

In [44]:
near_zero = test_data[(test_data['bbox_width'] < 3) | (test_data['bbox_height'] < 3)]
near_zero

,file_id,folder_path,file_name,image_width,image_height,class,cause_method,folder_path.1,ei_value,pl_value,...,pi_value,bbox,category_id,disease_status,name,bbox_x,bbox_y,bbox_width,bbox_height,object_status
30347,vl_normal_6622,./data_raw/test_image/test_normal/,normal_58_003_221127161748.jpg,1552,2106,normal,normal,./data_raw/test_image/test_normal/,0.03,6.00,...,5.7,"[1127.05, 1316.61, 0.47, 0.47]",4,N,stem,1127.05,1316.61,0.47,0.47,normal
64872,vl_normal_6622,./data_raw/test_image/test_normal/,normal_58_003_221127161748.jpg,1552,2106,normal,normal,./data_raw/test_image/test_normal/,0.03,6.00,...,5.7,"[1127.05, 1316.61, 0.47, 0.47]",4,N,stem,1127.05,1316.61,0.47,0.47,normal
94502,vl_blight_2948,./data_raw/test_image/test_blight/,blight_93_014_220917140549.jpg,828,1696,blight,blight_injection,./data_raw/test_image/test_blight/,0.00,6.46,...,6.2,"[494.83, 919.39, 0.76, 0.76]",4,Y,stem,494.83,919.39,0.76,0.76,abnormal


In [47]:
test_copy = test_data[(test_data['bbox_width'] > 1) & (test_data['bbox_height'] > 1)].copy()

print(len(test_copy))
print(len(test_data))

181340
181343


In [ ]:
#####################################################################################################################

In [13]:
# 직전에 해결한 문제: bbox y or height 값이 음수로 표기된 데이터 문제
    # 해당 컬럼 데이터에서 음수가 발견되는지 확인 -> 라벨 전처리 과정에서 bbox_y, bbox_height 값이 음수인 행 제외 처리함

# division zero 원인 검토 1: bbox width or height가 0 또는 음수값이 있는지 확인
    # 확인결과: 없음

zero_or_under = train_20[(train_20['bbox_width'] <= 0) | (train_20['bbox_height'] <= 0)]
zero_or_under

,file_id,folder_path,file_name,image_width,image_height,class,cause_method,ei_value,pl_value,el_value,pi_value,bbox,bbox_x,bbox_y,bbox_width,bbox_height,category_id,disease_status,name,object_status


In [16]:
# division zero 원인 검토 2: "x + width" or "y + height"가 실제 이미지의 너비, 높이를 초과하는지 확인
    # 확인결과: 전체 데이터 중 32개 행에서 초과 문제 발생 -> 이 행들을 삭제하는 내용을 추가하여 라벨 전처리를 다시 수행해야 함.
    # 추가 확인: "x + width" or "y + height가 실제 이미지의 너비, 높이와 동일한 경우도 식별됨(약 860개 행)
        # 대상 이미지에 해당 바운딩 박스를 그려보았을 때 단일 부위가 아닌 여러 부위들이 포함됨
        # 결론: 해당 데이터들도 이상치로 간주하고 제외해야 함.

size_check = train_20[['image_width', 'image_height', 'bbox_x', 'bbox_y', 'bbox_width', 'bbox_height']].copy()

size_check['x_width'] = size_check['bbox_x'] + size_check['bbox_width']
size_check['y_height'] = size_check['bbox_y'] + size_check['bbox_height']

In [18]:
over_check = size_check[(size_check['image_width'] < size_check['x_width']) | (size_check['image_height'] < size_check['y_height'])]

In [20]:
print(len(over_check))

32


In [21]:
train_100 = pd.read_csv('./data_preprocessed/label/train_label_20.csv', encoding='utf-8')

In [22]:
image_width = train_100['image_width']
image_height = train_100['image_height']
x = train_100['bbox_x']
y = train_100['bbox_y']
width = train_100['bbox_width']
height = train_100['bbox_height']

total_check = train_100[(image_width < (x+width)) | (image_height < (y+height))]

In [23]:
print(len(total_check))

32


In [42]:
train_20_test = train_20.copy()

width_same = train_20_test[image_width == (x+width)]
height_same = train_20_test[image_height == (y+height)]
width_height_same = train_20_test[(image_width == (x+width)) & (image_height == (y+height))]

print(len(width_same))
print(len(height_same))
print(len(width_height_same))
print(len(train_20_test) - len(width_height_same))
print(len(train_20_test))
print(width_same.head(1))

783
81
0
241664
241664
             file_id                           folder_path  \
506  tr_normal_47649  ./data_raw/train_image/train_normal/   

                          file_name  image_width  image_height   class  \
506  normal_55_008_221130091804.jpg          914          1585  normal   

    cause_method  ei_value  pl_value  el_value  pi_value  \
506       normal      0.07       6.2       3.2      11.0   

                              bbox  bbox_x  bbox_y  bbox_width  bbox_height  \
506  [355.5, 654.08, 29.51, 73.88]   355.5  654.08       29.51        73.88   

     category_id disease_status  name object_status  
506            4              N  stem        normal  


C:\Users\KM_AI_90\AppData\Local\Temp\ipykernel_10376\122370969.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  width_same = train_20_test[image_width == (x+width)]
C:\Users\KM_AI_90\AppData\Local\Temp\ipykernel_10376\122370969.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  height_same = train_20_test[image_height == (y+height)]
C:\Users\KM_AI_90\AppData\Local\Temp\ipykernel_10376\122370969.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  width_height_same = train_20_test[(image_width == (x+width)) & (image_height == (y+height))]


In [ ]:
# division zero 원인 검토 3: 바운딩 박스를 통한 크롭 영역이 0인 행이 있는지 확인하기
    # 바운딩 박스 좌표값의 유효성(바운딩 박스 길이와 높이가 실제 이미지의 길이, 높이를 초과하면 안됨)은 확인함
    # 데이터 유효성 확보임에도 여전히 문제가 발생한다면, 각 바운딩 박스 좌표가 대상 이미지 범위 내에서 실제로 좌표를 형성하는지를 확인해야 함




In [ ]:
# part image features save(.npz)
    # 두 방식으로 저장해서 비교해보기

train_20['part_features'] = features_list
part_features_array = np.array(train_20['part_features'].tolist())
np.savez_compressed('part_features_from_df.npz', features=part_features_array)

np.savez_compressed('part_features_saving_directly.npz', features=features_array)

In [6]:
#########################################################################################################

# part status classifying model build

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [7]:
class PartStatusClassifier(nn.Module):
    def __init__(self, input_size):
        super(PartStatusClassifier, self).__init__()
        self.layer1 = nn.Linear(input_size, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.layer2 = nn.Linear(512, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.layer2(out)
        out = self.sigmoid(out)

        return out

In [ ]:
## 샘플링한 데이터프레임에 별도로 샘플링 id 컬럼 부여
    ## bbox_id = file_id + _ + category_id
    ## sampling_id = 별도 부여(sam_#_index)
## 크롭 이미지 특징 추출 후 저장할 때 각 배열의 키 값으로 샘플링 id를 할당하기
## 위의 작업을 수행한 후 dataloader에 입력하기 위한 train, test 준비 함수 생성 및 실행
## 모델 훈련 및 평가 수행
## 다른 모듈 모델 정의 및 생성하기

In [8]:
# 데이터 매칭 구조
    # 라벨 데이터프레임: bbox_id, disease_status
    # 배열 딕셔너리: key = bbox_id, value = features array

# data_for_model 함수 구조
    # 데이터프레임 처리
        # disease_status 정수 라벨화, 리스트화
    # 배열 딕셔너리 처리
        # npz 로드
        # key, value 분리

    # dataloader 전달
    # return: train_loader, val_loader

def data_for_model(df, label_col, features_npz):
    # 데이터프레임 처리
    original_labels = df[label_col].copy()
    int_labels_tensor = torch.tensor(original_labels.map({'N' : 0, 'Y' : 1}).values, dtype=torch.float32)
    y_data = int_labels_tensor

    # features_npz 처리
    features_list = []

    for key in features_npz.files:
        cropped_img_array = features_npz[key]
        features_list.append(cropped_img_array)

    features_list_array = np.array(features_list)
    x_data = torch.tensor(features_list_array, dtype=torch.float32)

    return x_data, y_data

def create_dataloader(x_data, y_data, batch_size, shuffle_option):
    dataset = TensorDataset(x_data, y_data.view(-1, 1))
    created_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle_option)
    return created_loader
        



In [9]:

label_col = 'disease_status'
batch_size = 32

df_train = train_10.copy()
train_features_npz = np.load('./data_preprocessed/part_image_features/cropped_features_10_dict.npz')
df_test = test_data.copy()
test_features_npz = np.load('./data_preprocessed/part_image_features/vl_cropped_features_10_dict.npz')

x_train, y_train = data_for_model(df=df_train, label_col=label_col, features_npz=train_features_npz)
x_test, y_test = data_for_model(df=df_test, label_col=label_col, features_npz=test_features_npz)

train_loader = create_dataloader(x_data=x_train, y_data=y_train, batch_size=batch_size, shuffle_option=True)
test_loader = create_dataloader(x_data=x_test, y_data=y_test, batch_size=batch_size, shuffle_option=False)


AssertionError: Size mismatch between tensors

In [16]:
test_features_npz = np.load('./data_preprocessed/part_image_features/vl_cropped_features_10_dict.npz')

print(len(test_data))
print(len(test_features_npz))

18135
17573


In [20]:
test_key_list = list(test_features_npz.keys())

In [27]:
original = test_data['bbox_id'].tolist()
print(original)
print(test_key_list)

print(len(original))
print(len(test_key_list))

check = test_data.copy()

check = check[check['bbox_id'].isin(test_key_list)]

print(len(check))



['vl_normal_982_0', 'vl_normal_982_1', 'vl_normal_982_2', 'vl_normal_982_3', 'vl_normal_982_4', 'vl_normal_982_5', 'vl_normal_982_6', 'vl_normal_982_7', 'vl_normal_455_0', 'vl_normal_455_1', 'vl_normal_455_2', 'vl_normal_455_3', 'vl_normal_455_4', 'vl_normal_455_5', 'vl_normal_455_6', 'vl_normal_455_7', 'vl_normal_455_8', 'vl_normal_2207_0', 'vl_normal_2207_1', 'vl_normal_2207_2', 'vl_normal_2207_3', 'vl_normal_2207_4', 'vl_normal_2207_5', 'vl_normal_2207_6', 'vl_normal_6399_0', 'vl_normal_6399_1', 'vl_normal_6399_2', 'vl_normal_6399_3', 'vl_normal_6399_4', 'vl_normal_6399_5', 'vl_normal_4046_0', 'vl_normal_4046_1', 'vl_normal_4046_2', 'vl_normal_2416_0', 'vl_normal_2416_1', 'vl_normal_2416_2', 'vl_normal_2416_3', 'vl_normal_2416_4', 'vl_normal_999_0', 'vl_normal_999_1', 'vl_normal_999_2', 'vl_normal_999_3', 'vl_normal_999_4', 'vl_normal_912_0', 'vl_normal_912_1', 'vl_normal_912_2', 'vl_normal_912_3', 'vl_normal_912_4', 'vl_normal_912_5', 'vl_normal_7346_0', 'vl_normal_7346_1', 'vl_nor

In [ ]:
#######################################################################################################

# PartDiseaseClassifier model build

class PartDiseaseClassifier(nn.Module):
    def __init__(self, input_size):
        super(PartStatusClassifier, self).__init__()
        self.layer1 = nn.Linear(input_size, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.layer2 = nn.Linear(512, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.layer2(out)
        out = self.sigmoid(out)

        return out